![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

**Please restart the kernel. It's mandatory to get everything up and running.**

Take a look at the example data 

In [ ]:
import os
import zipfile
from SeisMonitor.utils4examples import clone_seismonitor_data

dataset = "../seismonitor_dataset"

clone_seismonitor_data(dataset,branch="dataset")
with zipfile.ZipFile(os.path.join(dataset,"downloads.zip"), 'r') as zip_ref:
    zip_ref.extractall(dataset)

downloads = os.path.join(dataset,"downloads")
stations = os.path.join(dataset,"stations")

print("Dataset dir: ",dataset)
print("Important folders in your dataset",["downloads","stations"])
print("#"*100)
print("downloads")
print("\t",os.listdir(downloads))
print("stations")
print("\t",os.listdir(stations))

cloning picker models

In [ ]:
from SeisMonitor.utils4examples import clone_seismonitor_data
models = "../picking_models"
clone_seismonitor_data(models,branch="models")

print("Models dir: ",models)
print("Important folders in your models",os.listdir(models))

# Picking with EQTransformer

In [ ]:
import os
from SeisMonitor.monitor.picker.ai import EQTransformer,EQTransformerObj
from SeisMonitor.monitor.picker import utils as piut

We create the object: EQTransformerObj, which contains the EQTransformer parameters.

In [ ]:
eqt_model = os.path.join(models,"EQTransformer_models","EqT_model.h5")
eqtobj = EQTransformerObj(eqt_path = None,
            model_path=eqt_model,
            n_processor = 6,
            overlap = 0.3,
            detection_threshold =0.1,
            P_threshold = 0.01,
            S_threshold = 0.01,
            batch_size = 20,
            number_of_plots = 0,
            plot_mode = 1 ) 

- EQTransformer is instanced with EQTransformerObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from EQTransformer
- We use eqt_picks_2_seismonitor_fmt to export seismic phases in specific format util for the rest of the workflow.

In [ ]:
out_dir = os.path.join("../eqt_picks")
result = os.path.join("../eqt_picks","seismonitor_picks.csv")

eqt = EQTransformer(eqtobj)
eqt.pick(downloads,stations,out_dir)
piut.eqt_picks_2_seismonitor_fmt(out_dir,downloads,result)

# Picking with PhaseNet

In [ ]:
import os
from SeisMonitor.monitor.picker.ai import PhaseNet,PhaseNetObj
from SeisMonitor.monitor.picker import utils as piut

Cloning PhaseNet repository to use the original PhaseNet pretrained model

In [ ]:
piut.git_clone_aipicker(name="PhaseNet",repo_dir=OUTPUT)

pnet_path = os.path.join(OUTPUT,"PhaseNet")
if os.path.isdir(pnet_path):
    print(True)

We create the object: PhaseNetObj, which contains the PhaseNet parameters.

In [ ]:
pnetobj = PhaseNetObj(pnet_path,
            P_threshold=0.7, S_threshold=0.6,
            batch_size=100
            ) 

- PhaseNet is instanced with PhaseNetObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from PhaseNet

In [ ]:
mseed_storage = os.path.join(download_folder,"archive")
json_dir = os.path.join(download_folder,"stations")
out_dir = os.path.join(OUTPUT,"picks","pnet")
result = os.path.join(OUTPUT,"picks","pnet","seismonitor_picks.csv")

pnet = PhaseNet(pnetobj)
pnet.pick(mseed_storage,json_dir,out_dir)